In [119]:
import pandas as pd
import numpy as np

In [120]:
training = pd.read_csv('cybersecurity_training.csv', delimiter="|", header=[0])

In [121]:
training.head()

,alert_ids,client_code,notified,categoryname,ip,ipcategory_name,ipcategory_scope,parent_category,grandparent_category,overallseverity,...,thrcnt_week,thrcnt_day,p6,p9,p5m,p5w,p5d,p8m,p8w,p8d
0,Nhq,DPM,0,Attack,YT.LB.32.21,INTERNET,Internet,7,A,3,...,4160,675,1,0,2,1,1,1,1,1
1,XZt,FIN,0,Exploit,192.SL.UK.94,PRIV-192,Private network,1,A,5,...,9,2,4,12,3,2,2,2,1,1
2,bBz,CHP,0,Attack,YT.LB.38.21,INTERNET,Internet,7,A,4,...,3788,628,1,0,2,2,1,2,2,1
3,ZNr,HPS,0,Attack,JX.NY.13.20,INTERNET,Internet,7,A,4,...,565,96,0,0,2,2,2,2,2,2
4,poV,OSC,0,Attack,YT.LB.32.21,INTERNET,Internet,7,A,4,...,2790,632,1,0,1,1,1,1,1,1


In [122]:
training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39427 entries, 0 to 39426
Data columns (total 63 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   alert_ids                 39427 non-null  object 
 1   client_code               39427 non-null  object 
 2   notified                  39427 non-null  int64  
 3   categoryname              39427 non-null  object 
 4   ip                        39427 non-null  object 
 5   ipcategory_name           39427 non-null  object 
 6   ipcategory_scope          39427 non-null  object 
 7   parent_category           39427 non-null  int64  
 8   grandparent_category      39427 non-null  object 
 9   overallseverity           39427 non-null  int64  
 10  timestamp_dist            39427 non-null  int64  
 11  start_hour                39427 non-null  int64  
 12  start_minute              39427 non-null  int64  
 13  start_second              39427 non-null  int64  
 14  weekda

In [123]:
training["categoryname"].unique()

array(['Attack', 'Exploit', 'Suspicious Reputation',
       'Control and Maintain', 'Reconnaissance', 'Malicious Activity',
       'Suspicious Network Activity', 'Attack Preparation', 'Compromise',
       'Suspicious Account Activity', 'To Be Determined'], dtype=object)

In [124]:
for column in training.columns:
        if training[column].isna().sum() > 0:
            print(column, training[column].isna().sum())

n1 32295
n2 32295
n3 32295
n4 32295
n5 32295
n6 32295
n7 32295
n8 32295
n9 32295
n10 32295
score 32295


In [125]:
def fillmissing(dataset):
    for column in dataset.columns:
        if dataset[column].isna().sum() > 0:
            dataset[column].fillna(0, inplace=True)
    return dataset

In [126]:
training = fillmissing(training)

In [127]:
training = training.drop('alert_ids', axis=1)
training = training.drop("client_code", axis=1)
training = training.drop("ip", axis=1)

In [128]:
training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39427 entries, 0 to 39426
Data columns (total 60 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   notified                  39427 non-null  int64  
 1   categoryname              39427 non-null  object 
 2   ipcategory_name           39427 non-null  object 
 3   ipcategory_scope          39427 non-null  object 
 4   parent_category           39427 non-null  int64  
 5   grandparent_category      39427 non-null  object 
 6   overallseverity           39427 non-null  int64  
 7   timestamp_dist            39427 non-null  int64  
 8   start_hour                39427 non-null  int64  
 9   start_minute              39427 non-null  int64  
 10  start_second              39427 non-null  int64  
 11  weekday                   39427 non-null  object 
 12  correlatedcount           39427 non-null  int64  
 13  n1                        39427 non-null  float64
 14  n2    

In [129]:
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder()

In [130]:
category_columns = ("categoryname", "ipcategory_name", "ipcategory_scope", "grandparent_category", "weekday", "dstipcategory_dominate", "srcipcategory_dominate")

In [131]:
for column in category_columns:
    training = training.join(
        pd.DataFrame.sparse.from_spmatrix(cat_encoder.fit_transform(training[[column]]), columns=[column+ str(i) for i in (training[column].unique())]))
    training = training.drop(column, axis=1)

In [132]:
training.head()

,notified,parent_category,overallseverity,timestamp_dist,start_hour,start_minute,start_second,correlatedcount,n1,n2,...,dstipcategory_dominateCURR_NET,dstipcategory_dominateLOOPBACK,dstipcategory_dominateMULTICAST,srcipcategory_dominatePRIV-192,srcipcategory_dominatePRIV-10,srcipcategory_dominateINTERNET,srcipcategory_dominatePRIV-172,srcipcategory_dominatePRIV-CGN,srcipcategory_dominateLOOPBACK,srcipcategory_dominateLINK-LOCAL
0,0,7,3,65684,8,14,34,69,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0,1,5,1188030,1,0,2,5302,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0,7,4,43716,14,44,5,346,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0,7,4,0,20,33,40,1,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,7,4,2401,14,36,11,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [133]:
training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39427 entries, 0 to 39426
Columns: 102 entries, notified to srcipcategory_dominateLINK-LOCAL
dtypes: Sparse[float64, 0](49), float64(11), int64(42)
memory usage: 19.1 MB


In [134]:
file_name = "onehot_train.pkl"
training.to_pickle(file_name)